In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, ShuffleSplit, KFold
from sklearn import metrics
import keras
from keras.layers import Dense, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.layers import TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

2025-02-14 14:32:08.404017: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 14:32:08.404654: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 14:32:08.406979: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 14:32:08.414554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739523728.426809   58393 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739523728.43

In [4]:
TIMESTEPS = 128
BATCH_SIZE = 16
PFAM2VEC_DIMENSIONS = 100

In [5]:
labels = pd.read_csv("MIBiG.classes.csv").set_index('contig_id')
NUM_LABELS = labels.shape[1]
labels

,Alkaloid,NRP,Other,Polyketide,RiPP,Saccharide,Terpene
contig_id,,,,,,,
BGC0000001.1,0,0,0,1,0,0,0
BGC0000002.1,0,0,0,1,0,0,0
BGC0000003.1,0,0,0,1,0,0,0
BGC0000004.1,0,0,0,1,0,0,0
BGC0000005.1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
BGC0001826.1,0,1,0,0,0,0,0
BGC0001827.1,0,0,0,1,0,0,0
BGC0001828.1,0,0,0,1,0,0,0


In [6]:
labels.sum()


Alkaloid       54
NRP           603
Other         247
Polyketide    849
RiPP          261
Saccharide    187
Terpene       167
dtype: int64

In [7]:
# Count the total number of BGCs per class
label_counts = labels.sum()

# Count the number of BGCs that belong to multiple classes
overlap_counts = labels[labels.sum(axis=1) > 1].sum()

# Count the number of unique overlaps
num_overlapping_bgcs = (labels.sum(axis=1) > 1).sum()

print("Total counts per class:")
print(label_counts)

print("\nTotal counts of overlapping BGCs per class:")
print(overlap_counts)

print(f"\nTotal number of BGCs with overlaps: {num_overlapping_bgcs}")


Total counts per class:
Alkaloid       54
NRP           603
Other         247
Polyketide    849
RiPP          261
Saccharide    187
Terpene       167
dtype: int64

Total counts of overlapping BGCs per class:
Alkaloid       12
NRP           265
Other          11
Polyketide    308
RiPP            1
Saccharide     54
Terpene        36
dtype: int64

Total number of BGCs with overlaps: 337


In [8]:
import pandas as pd

# Load the labels
labels = pd.read_csv("MIBiG.classes.csv").set_index('contig_id')

# Count occurrences of each class
label_counts = labels.sum()

# Find BGCs that belong to multiple classes
overlapping_bgcs = labels[labels.sum(axis=1) > 1]

# Count how often each class overlaps with others
overlap_matrix = overlapping_bgcs.T.dot(overlapping_bgcs)

print("Total counts per class:")
print(label_counts)

print("\nTotal number of BGCs with overlaps:", len(overlapping_bgcs))

print("\nOverlap matrix (how often classes co-occur):")
print(overlap_matrix)


Total counts per class:
Alkaloid       54
NRP           603
Other         247
Polyketide    849
RiPP          261
Saccharide    187
Terpene       167
dtype: int64

Total number of BGCs with overlaps: 337

Overlap matrix (how often classes co-occur):
            Alkaloid  NRP  Other  Polyketide  RiPP  Saccharide  Terpene
Alkaloid          12    6      0           3     0           0        6
NRP                6  265      5         251     1          10        2
Other              0    5     11           4     0           6        0
Polyketide         3  251      4         308     0          40       22
RiPP               0    1      0           0     1           0        0
Saccharide         0   10      6          40     0          54        7
Terpene            6    2      0          22     0           7       36


In [9]:
# Find BGCs where the sum of ones (labels) per row is greater than 3
bgcs_with_more_than_3_labels = labels[labels.sum(axis=1) >= 3]

# Count how many such rows exist
num_bgcs_with_more_than_3_labels = bgcs_with_more_than_3_labels.shape[0]

# Display the rows
print(f"Number of BGCs with more than 3 labels: {num_bgcs_with_more_than_3_labels}")
print(bgcs_with_more_than_3_labels)


Number of BGCs with more than 3 labels: 13
              Alkaloid  NRP  Other  Polyketide  RiPP  Saccharide  Terpene
contig_id                                                                
BGC0000119.1         0    0      1           1     0           1        0
BGC0000269.1         0    0      1           1     0           1        0
BGC0000297.1         0    1      0           1     0           1        0
BGC0000963.1         0    1      0           1     0           1        0
BGC0000978.1         1    1      0           1     0           0        0
BGC0001003.1         0    1      0           1     0           1        0
BGC0001046.1         0    1      0           1     0           1        0
BGC0001048.1         0    1      0           1     0           1        0
BGC0001056.1         0    1      1           1     0           0        0
BGC0001058.1         0    1      0           1     0           1        0
BGC0001072.1         0    0      1           1     0           1     

In [10]:
from gensim.models import Word2Vec

# Load the model correctly using gensim
pfam2vec_bin = Word2Vec.load('Word2Vec/word2vec_model.bin')

# Convert to DataFrame
pfam2vec = pd.DataFrame(pfam2vec_bin.wv.vectors, index=pfam2vec_bin.wv.index_to_key)

# Show first two rows
pfam2vec


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
PF00005,0.233514,0.052725,0.033491,0.240617,-0.052133,-0.383191,0.097324,0.307650,-0.516893,-0.119899,...,-0.130947,-0.349238,-0.052996,0.266614,0.556849,-0.025646,0.321105,-0.287103,0.029798,0.149110
PF07690,0.084144,0.282901,-0.045280,0.076797,-0.022403,-0.326592,0.109536,0.147560,0.070727,-0.395535,...,0.271868,-0.034904,-0.008765,-0.342772,-0.019638,-0.114983,-0.203931,-0.463244,0.079770,0.186862
PF13304,0.111681,0.023277,0.181347,0.151265,-0.179347,-0.140607,0.081777,0.406897,-0.509895,-0.132071,...,-0.100485,-0.220790,-0.087275,0.204365,0.523665,-0.074779,0.398022,-0.301283,0.035700,0.046756
PF00072,-0.303923,-0.144928,0.174792,0.111320,-0.050851,-0.241886,0.114559,0.242707,0.023629,-0.339719,...,0.259481,-0.173986,-0.335270,-0.064067,0.594384,-0.297773,-0.032209,-0.532959,0.102281,0.016370
PF00528,-0.073311,0.541115,-0.048413,-0.371761,0.181419,0.124935,0.017770,0.555671,-0.512254,-0.206273,...,0.077985,-0.153859,0.116167,0.072122,0.970655,0.079777,0.441720,-0.321656,0.043775,0.187739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PF00518,-0.070158,-0.065413,0.019068,0.022858,0.069239,-0.179477,0.119020,0.293031,-0.051709,-0.138381,...,0.104844,-0.152967,0.031874,0.091196,-0.031648,-0.022964,0.043654,-0.201326,0.148341,0.097215
PF07033,-0.010519,0.020789,-0.006451,-0.060890,-0.009975,-0.159964,0.054209,0.259067,-0.032121,-0.150636,...,-0.011022,0.020476,0.119876,0.037193,0.120064,-0.122264,0.046126,-0.287325,0.053798,-0.106282
PF15873,-0.110694,-0.012107,0.059395,-0.005941,0.106874,-0.165131,-0.093995,0.093994,0.076426,-0.070573,...,0.111926,-0.116411,0.112473,0.085117,0.157332,0.065394,0.070263,-0.179310,0.149701,-0.101817
PF15049,-0.126276,0.086068,0.140609,-0.026558,0.106151,-0.213212,-0.017060,0.259988,0.099858,-0.165359,...,-0.014387,-0.078912,0.023920,0.009382,0.090026,0.048761,0.020485,-0.080630,0.066559,-0.094392


In [11]:
domains = pd.read_csv('MIBiG.pfam.tsv', delimiter='\t')
domains

,sequence_id,protein_id,gene_start,gene_end,gene_strand,pfam_id,in_cluster
0,BGC0000001.1,AEK75490.1,0,1083,1,PF02353,1
1,BGC0000001.1,AEK75490.1,0,1083,1,PF01135,1
2,BGC0000001.1,AEK75490.1,0,1083,1,PF01269,1
3,BGC0000001.1,AEK75490.1,0,1083,1,PF13489,1
4,BGC0000001.1,AEK75490.1,0,1083,1,PF01596,1
...,...,...,...,...,...,...,...
96407,BGC0001833.1,AYA44686.1,0,15051,1,PF13193,1
96408,BGC0001833.1,AYA44686.1,0,15051,1,PF00668,1
96409,BGC0001833.1,AYA44686.1,0,15051,1,PF00550,1
96410,BGC0001833.1,AYA44686.1,0,15051,1,PF00975,1


In [12]:
contig_ids = domains['sequence_id'].unique()
contig_ids = list(np.intersect1d(contig_ids, labels.index))
len(contig_ids)

1978

In [13]:
contig_ids[-1]

'BGC0001830.1'

In [14]:
def make_batch_size_divisible(vectors):
    missing_to_divisible = BATCH_SIZE - (len(vectors) % BATCH_SIZE)
    shape = list(vectors.shape)
    shape[0] = missing_to_divisible
    print(missing_to_divisible)
    return np.concatenate([vectors, np.zeros(shape=shape)])


In [15]:
y_samples = labels.reindex(contig_ids).values
y_samples = pd.DataFrame(make_batch_size_divisible(y_samples), columns=labels.columns)
print(y_samples.shape)
y_samples

6
(1984, 7)


,Alkaloid,NRP,Other,Polyketide,RiPP,Saccharide,Terpene
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1979,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1981,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1982,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
domain_vectors = pfam2vec.reindex(domains['pfam_id'])
domain_vectors['contig_id'] = domains['sequence_id'].values
sample_vectors = domain_vectors.groupby('contig_id')
X_samples = [sample_vectors.get_group(contig_id).drop('contig_id', axis=1).dropna() for contig_id in contig_ids]
X_samples = pad_sequences(X_samples, maxlen=TIMESTEPS, dtype=np.float64, padding='post', truncating='post')
X_samples = make_batch_size_divisible(X_samples)
#X_samples = X_samples.reshape(-1, BATCH_SIZE, TIMESTEPS, PFAM2VEC_DIMENSIONS)
print(X_samples.shape)
print('First sample shape:', X_samples[0].shape)
X_samples[0]

6
(1984, 128, 100)
First sample shape: (128, 100)


array([[ 0.25916222,  0.16687518,  0.45584118, ...,  0.23456158,
         0.26959524, -0.38498661],
       [ 0.00194787,  0.24177134,  0.01988547, ...,  0.225835  ,
         0.05770348, -0.13457496],
       [ 0.36128652,  0.64170277,  0.63306242, ...,  0.2202387 ,
         0.71887439, -0.57087594],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [17]:
len(X_samples[0])

128

In [89]:
i = 0
maxi = 0
avg = 0
extra = 0
for contig_id, contig_bgcs in sample_vectors:

    maxi = max(maxi, len(contig_bgcs))
    avg += len(contig_bgcs)
    if(len(contig_bgcs) > 128): extra += 1
    i += 1

print(maxi, avg/i, extra)

317 48.59475806451613 142


In [99]:
major_bgc_ids = pd.Series([contig_id.split('.')[0] for contig_id in contig_ids])
major_to_minor_ids = pd.Series(range(0, len(contig_ids)), index=major_bgc_ids)
major_to_minor_ids

BGC0000001       0
BGC0000002       1
BGC0000003       2
BGC0000004       3
BGC0000005       4
              ... 
BGC0001826    1973
BGC0001827    1974
BGC0001828    1975
BGC0001829    1976
BGC0001830    1977
Length: 1978, dtype: int64

In [100]:
unique_major_ids = major_bgc_ids.unique()
len(unique_major_ids)

1806

In [115]:
from keras.layers import InputLayer

def create_lstm(stacked_sizes=[], batched=True):
    model = Sequential()
    
    if batched:
        # Add InputLayer with batch_input_shape
        model.add(InputLayer(batch_input_shape=(BATCH_SIZE, TIMESTEPS, PFAM2VEC_DIMENSIONS)))
    else:
        # Add InputLayer with input_shape (no batch size)
        model.add(InputLayer(input_shape=(TIMESTEPS, PFAM2VEC_DIMENSIONS)))

    # Add Bidirectional LSTM layer
    model.add(Bidirectional(
        layer=LSTM(
            units=16,
            return_sequences=False,
            dropout=0.2,
            recurrent_dropout=0.2,
            stateful=False
        )
    ))

    # Add stacked LSTM layers if specified
    for size in stacked_sizes:
        model.add(Bidirectional(LSTM(units=size, return_sequences=False, stateful=False)))

    # Add the output layer
    model.add(Dense(NUM_LABELS, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=[])
    return model

In [112]:
def create_conv1D(stacked_sizes=[], batched=True):
    model = Sequential()
    args = {}
    if batched:
        args['batch_input_shape'] = (BATCH_SIZE, TIMESTEPS, 100)
    else:
        args['input_shape'] = (TIMESTEPS, 100)
    model.add(Conv1D(128, 3, activation='relu', **args))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=[])
    return model

In [157]:
def evaluate_1(create_model, epochs=200):
    splitter = KFold(n_splits=5, shuffle=True, random_state=0)

    scores = []
    accuracies = []
    best_accuracy = 0  # Track the best accuracy
    best_model = None  # Track the best model

    for fold, (id_train_idx, id_test_idx) in enumerate(splitter.split(unique_major_ids)):
        train_major_ids, test_major_ids = unique_major_ids[id_train_idx], unique_major_ids[id_test_idx]
        train_minor_ids, test_minor_ids = major_to_minor_ids.loc[train_major_ids], major_to_minor_ids.loc[test_major_ids]
        X_train, X_test = X_samples[train_minor_ids], X_samples[test_minor_ids]
        y_train, y_test = y_samples.loc[train_minor_ids].values, y_samples.loc[test_minor_ids].values
        X_train = make_batch_size_divisible(X_train)
        y_train = make_batch_size_divisible(y_train)
        print('Train:', len(X_train), 'Test:', len(X_test))

        # Create and train the model
        train_model = create_model(batched=True)
        train_model.fit(X_train, y_train, epochs=epochs, verbose=2, batch_size=BATCH_SIZE, shuffle=False)

        # Create a model for evaluation
        model = create_model(batched=False)
        model.set_weights(train_model.get_weights())

        pred = (model.predict(X_test) > 0.5).astype(np.int64)
        aucs = metrics.roc_auc_score(y_test, pred, average=None)
        precisions = metrics.precision_score(y_test, pred, average=None)
        recalls = metrics.recall_score(y_test, pred, average=None)
        accuracy = metrics.accuracy_score(y_test, pred)
        accuracies.append(accuracy)

        # Append scores for each class
        scores += [{'Precision': p, 'Recall': r, 'AUC': a, 'Class': c, 'Samples': int(sum(y_samples[c]))} 
                   for p, r, a, c in zip(precisions, recalls, aucs, y_samples.columns)]

        # Track the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = train_model  # Save the best model

    # Save the best model after all folds are complete
    if best_model is not None:
        best_model.save("2.best_model.h5")
        print(f"Best model saved with accuracy: {best_accuracy}")

    scores = pd.DataFrame(scores).groupby('Class').mean().sort_values('Samples', ascending=False)
    print('Accuracy (exact match)', np.mean(accuracies))
    print(scores.mean())
    return scores

In [158]:
scores = evaluate_1(create_lstm, epochs=100)

13
13
Train: 1584 Test: 407
Epoch 1/100
99/99 - 6s - 58ms/step - loss: 2.1412
Epoch 2/100
99/99 - 4s - 41ms/step - loss: 1.9205
Epoch 3/100
99/99 - 4s - 42ms/step - loss: 1.7134
Epoch 4/100
99/99 - 4s - 42ms/step - loss: 1.5816
Epoch 5/100
99/99 - 4s - 40ms/step - loss: 1.4770
Epoch 6/100
99/99 - 4s - 39ms/step - loss: 1.4113
Epoch 7/100
99/99 - 4s - 42ms/step - loss: 1.3426
Epoch 8/100
99/99 - 4s - 40ms/step - loss: 1.3109
Epoch 9/100
99/99 - 4s - 41ms/step - loss: 1.2837
Epoch 10/100
99/99 - 4s - 42ms/step - loss: 1.2485
Epoch 11/100
99/99 - 4s - 44ms/step - loss: 1.2379
Epoch 12/100
99/99 - 4s - 41ms/step - loss: 1.2600
Epoch 13/100
99/99 - 4s - 40ms/step - loss: 1.2109
Epoch 14/100
99/99 - 4s - 40ms/step - loss: 1.2388
Epoch 15/100
99/99 - 4s - 40ms/step - loss: 1.2248
Epoch 16/100
99/99 - 4s - 43ms/step - loss: 1.2026
Epoch 17/100
99/99 - 4s - 43ms/step - loss: 1.2237
Epoch 18/100
99/99 - 4s - 40ms/step - loss: 1.2021
Epoch 19/100
99/99 - 4s - 40ms/step - loss: 1.2255
Epoch 20/100

/home/ultron/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
14
14
Train: 1600 Test: 392
Epoch 1/100
100/100 - 7s - 65ms/step - loss: 2.0727
Epoch 2/100
100/100 - 5s - 46ms/step - loss: 1.8916
Epoch 3/100
100/100 - 5s - 50ms/step - loss: 1.6779
Epoch 4/100
100/100 - 5s - 50ms/step - loss: 1.5217
Epoch 5/100
100/100 - 5s - 49ms/step - loss: 1.4244
Epoch 6/100
100/100 - 5s - 48ms/step - loss: 1.3840
Epoch 7/100
100/100 - 5s - 48ms/step - loss: 1.3210
Epoch 8/100
100/100 - 5s - 48ms/step - loss: 1.2936
Epoch 9/100
100/100 - 4s - 45ms/step - loss: 1.2873
Epoch 10/100
100/100 - 5s - 47ms/step - loss: 1.2812
Epoch 11/100
100/100 - 5s - 47ms/step - loss: 1.2600
Epoch 12/100
100/100 - 5s - 52ms/step - loss: 1.2793
Epoch 13/100
100/100 - 5s - 50ms/step - loss: 1.2503
Epoch 14/100
100/100 - 5s - 45ms/step - loss: 1.2498
Epoch 15/100
100/100 - 5s - 45ms/step - loss: 1.2662
Epoch 16/100
100/100 - 6s - 58ms/step - loss: 1.2402
Epoch 17/100
100/100 - 5s - 53ms/step - loss: 1.2490
Epoch 18/100
100/100 - 5s - 47ms/step - 

/home/ultron/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step
7
7
Train: 1616 Test: 369
Epoch 1/100
101/101 - 9s - 84ms/step - loss: 2.0473
Epoch 2/100
101/101 - 6s - 54ms/step - loss: 1.8459
Epoch 3/100
101/101 - 5s - 51ms/step - loss: 1.6458
Epoch 4/100
101/101 - 5s - 50ms/step - loss: 1.5493
Epoch 5/100
101/101 - 5s - 47ms/step - loss: 1.5115
Epoch 6/100
101/101 - 5s - 48ms/step - loss: 1.4293
Epoch 7/100
101/101 - 5s - 47ms/step - loss: 1.3739
Epoch 8/100
101/101 - 5s - 48ms/step - loss: 1.3341
Epoch 9/100
101/101 - 5s - 49ms/step - loss: 1.2983
Epoch 10/100
101/101 - 5s - 47ms/step - loss: 1.2764
Epoch 11/100
101/101 - 5s - 49ms/step - loss: 1.2383
Epoch 12/100
101/101 - 5s - 54ms/step - loss: 1.2291
Epoch 13/100
101/101 - 7s - 68ms/step - loss: 1.2238
Epoch 14/100
101/101 - 6s - 57ms/step - loss: 1.2073
Epoch 15/100
101/101 - 5s - 54ms/step - loss: 1.2116
Epoch 16/100
101/101 - 5s - 49ms/step - loss: 1.2152
Epoch 17/100
101/101 - 5s - 47ms/step - loss: 1.1828
Epoch 18/100
101/101 - 5s - 45ms/step - lo

...

/home/ultron/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step
2
2
Train: 1568 Test: 412
Epoch 1/100
98/98 - 5s - 55ms/step - loss: 2.0356
Epoch 2/100
98/98 - 4s - 39ms/step - loss: 1.8687
Epoch 3/100
98/98 - 4s - 42ms/step - loss: 1.6726
Epoch 4/100
98/98 - 4s - 42ms/step - loss: 1.5686
Epoch 5/100
98/98 - 4s - 38ms/step - loss: 1.5136
Epoch 6/100
98/98 - 4s - 38ms/step - loss: 1.5049
Epoch 7/100
98/98 - 4s - 36ms/step - loss: 1.4300
Epoch 8/100
98/98 - 4s - 38ms/step - loss: 1.4437
Epoch 9/100
98/98 - 4s - 37ms/step - loss: 1.4036
Epoch 10/100
98/98 - 4s - 36ms/step - loss: 1.4032
Epoch 11/100
98/98 - 4s - 36ms/step - loss: 1.3766
Epoch 12/100
98/98 - 4s - 36ms/step - loss: 1.3333
Epoch 13/100
98/98 - 4s - 38ms/step - loss: 1.3397
Epoch 14/100
98/98 - 4s - 39ms/step - loss: 1.3034
Epoch 15/100
98/98 - 4s - 37ms/step - loss: 1.3021
Epoch 16/100
98/98 - 4s - 37ms/step - loss: 1.2724
Epoch 17/100
98/98 - 4s - 41ms/step - loss: 1.2515
Epoch 18/100
98/98 - 4s - 42ms/step - loss: 1.2472
Epoch 19/100
98/98 - 4s -

/home/ultron/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step


Best model saved with accuracy: 0.6422764227642277
Accuracy (exact match) 0.628609405005581
Precision      0.746076
Recall         0.570213
AUC            0.770637
Samples      331.857143
dtype: float64


In [159]:
scores

,Precision,Recall,AUC,Samples
Class,,,,
Polyketide,0.879722,0.871058,0.892950,824.0
NRP,0.869225,0.500820,0.734256,600.0
RiPP,0.913922,0.837501,0.913338,254.0
Other,0.646218,0.491168,0.726341,246.0
Saccharide,0.874016,0.627468,0.808512,187.0
Terpene,0.699433,0.583790,0.781534,158.0
Alkaloid,0.340000,0.079683,0.537527,54.0


In [160]:
scores.mean()

Precision      0.746076
Recall         0.570213
AUC            0.770637
Samples      331.857143
dtype: float64